In [3]:
import pandas as pd
from pathlib import Path

base_path = Path("D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa")

for split in ["train", "test", "dev"]:
    tsv_file = base_path / f"{split}.tsv"
    csv_file = base_path / f"{split}.csv"

    try:
        df = pd.read_csv(tsv_file, sep="\t", quoting=3, on_bad_lines='skip', encoding="utf-8")
        df.to_csv(csv_file, index=False, encoding="utf-8")
        print(f"✅ Converted {tsv_file} to {csv_file}")
    except Exception as e:
        print(f"❌ Error while converting {tsv_file}: {e}")


✅ Converted D:\projects\SPEECH\cv-corpus-21.0-2025-03-14\fa\train.tsv to D:\projects\SPEECH\cv-corpus-21.0-2025-03-14\fa\train.csv
✅ Converted D:\projects\SPEECH\cv-corpus-21.0-2025-03-14\fa\test.tsv to D:\projects\SPEECH\cv-corpus-21.0-2025-03-14\fa\test.csv
✅ Converted D:\projects\SPEECH\cv-corpus-21.0-2025-03-14\fa\dev.tsv to D:\projects\SPEECH\cv-corpus-21.0-2025-03-14\fa\dev.csv


In [10]:
from datasets import load_dataset, Audio, concatenate_datasets
import os

clips_dir = "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/clips"
data_files = {
    "train": "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/train.csv",
    "test":  "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/test.csv",
    "validation": "D:/projects/SPEECH/cv-corpus-21.0-2025-03-14/fa/dev.csv"
}

dataset = load_dataset("csv", data_files=data_files)

def add_full_path(example):
    example["path"] = os.path.join(clips_dir, example["path"])
    return example

dataset = dataset.map(add_full_path)

dataset = dataset.cast_column("path", Audio(sampling_rate=16000))

for split in dataset:
    dataset[split] = dataset[split].rename_columns({
        "path": "audio",
        "sentence": "text"
    })

for split in dataset:
    dataset[split] = dataset[split].remove_columns(
        [col for col in dataset[split].column_names if col not in ["audio", "text"]]
    )

train_dataset = concatenate_datasets([dataset["train"], dataset["validation"]])
test_dataset = dataset["test"]

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Train samples: 40397
Test samples: 10668
